In [1]:
from scipy import stats
import os
import pickle
from collections import OrderedDict
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import matplotlib.gridspec as gridspec
import operator
from plotly import graph_objs as go
import pandas as pd
import plotly

In [2]:
stateCodes=pickle.load(open("stateCode.pickle","rb"))
paths=[str(i)+'/' for i in stateCodes]

yearMap=OrderedDict()
yearMapRev=OrderedDict()
count=0
for year in range(1980,2018):
    yearMap[str(year)]=count
    yearMapRev[count]=str(year)
    count+=1
    
carbA=[6,41,53]
carbB=[9,10,23,24,25,34,36,42,44,50,11]
noState = ['"Guam"','"Puerto Rico"','"Hawaii"','"Virgin Islands"','"Alaska"','"Country Of Mexico"']
nonCarb = [i for i in stateCodes if i not in carbA+carbB and stateCodes[i] not in noState]

def getSlope(data):
    t1 = list(range(len(data)))
    t2 = [data[i] for i in data]
    tx1,tx2=[],[]
    for i,j in zip(t1,t2):
        if j!=None:
            tx1.append(i)
            tx2.append(j)
    try:
        r1 = stats.linregress(tx1,tx2)[0]
    except:
        r1= np.nan
    return r1

def setplot(plt,ax):
   

    ax.spines["top"].set_visible(False)  
    ax.spines["bottom"].set_visible(False)  
    ax.spines["right"].set_visible(False)  
    ax.spines["left"].set_visible(False)

    ax.get_xaxis().tick_bottom()    
    ax.get_yaxis().tick_left()  
    return plt,ax

def sortByValue(tempx):
    by_key = sorted(tempx.items(), key=lambda d: d[0])
    x = sorted(by_key, key=lambda d: d[1], reverse=False)
    return x

In [3]:
monitoringSites =pickle.load(open("monitoringSites.pickle",'rb'))
clusteringCoef =pickle.load(open("clusteringCoef.pickle",'rb'))

In [4]:
data=OrderedDict()
for codes in monitoringSites:
    x,y=[],[]
    r1, r2 = getSlope(monitoringSites[codes]), getSlope(clusteringCoef[codes])
    for i in monitoringSites[codes]:
        if monitoringSites[codes][i]!=None and clusteringCoef[codes][i]!=None:
            x.append(monitoringSites[codes][i])
            y.append(clusteringCoef[codes][i])
    data[codes]=[r1,r2,np.corrcoef(x,y)[0][1]]
    
regionCC= pickle.load(open("clusteringCoefRegion.pickle",'rb'))
regionMon=pickle.load(open("monitoringSitesRegion.pickle",'rb'))    

c:\program files (x86)\python36-32\lib\site-packages\numpy\lib\function_base.py:1110: RuntimeWarning:

Mean of empty slice.

c:\program files (x86)\python36-32\lib\site-packages\numpy\core\_methods.py:73: RuntimeWarning:

invalid value encountered in true_divide

c:\program files (x86)\python36-32\lib\site-packages\numpy\lib\function_base.py:3159: RuntimeWarning:

Degrees of freedom <= 0 for slice

c:\program files (x86)\python36-32\lib\site-packages\numpy\lib\function_base.py:3093: RuntimeWarning:

divide by zero encountered in double_scalars

c:\program files (x86)\python36-32\lib\site-packages\numpy\lib\function_base.py:3093: RuntimeWarning:

invalid value encountered in multiply



In [5]:
df = pd.DataFrame()
state_abbre = {}
state_name = {}
f = open("states_and_counties.csv","r")
f.readline()
state_code_list = []
for l in f:
    row = l.strip().split(",")
    state_code, abbre, name = row[0].replace('"',''), row[2].replace('"',''), row[1].replace('"','')
    if state_code!='CC':
        if str(int(state_code)) not in state_abbre:
            state_abbre[str(int(state_code))] = abbre
            state_name[str(int(state_code))] = name
            state_code_list.append(str(int(state_code)))
df['State_Code'] = state_code_list
df['State_Abbre'] = [state_abbre[code] for code in state_code_list]
df['State_Name'] =  [state_name[code] for code in state_code_list]

In [6]:
# monitoring_sites_slope, clus_coeff_slope, correlation = {}, {}, {}
# for stateCode in data:
#     monitoring_sites_slope[state_abbre[stateCode]] = data[stateCode][0]
#     clus_coeff_slope[state_abbre[stateCode]] = data[stateCode][1]
#     correlation[state_abbre[stateCode]] = data[stateCode][2]

monitoring_sites_slope, clus_coeff_slope, correlation = [], [],[]
for index, row in df.iterrows():
    stateCode = row['State_Code']
    monitoring_sites_slope.append(data[stateCode][0])
    clus_coeff_slope.append(data[stateCode][1])
    correlation.append(data[stateCode][2])
    
df['Monitoring_Site'] = monitoring_sites_slope
df['Clus_Coef'] = clus_coeff_slope
df['Corr'] = correlation

In [7]:
df_na = df.dropna()

In [8]:
df_na

,State_Code,State_Abbre,State_Name,Monitoring_Site,Clus_Coef,Corr
0,1,AL,Alabama,0.510669,0.002674,0.520218
2,4,AZ,Arizona,1.046066,0.005943,0.693975
3,5,AR,Arkansas,0.209760,0.023208,0.760976
4,6,CA,California,1.242258,0.001101,0.308645
5,8,CO,Colorado,0.992341,-0.003787,-0.713072
6,9,CT,Connecticut,0.049021,-0.000891,-0.540416
7,10,DE,Delaware,0.072874,0.003111,0.431192
8,11,DC,District Of Columbia,0.023963,0.025311,0.903575
9,12,FL,Florida,1.255717,0.004193,0.744171
10,13,GA,Georgia,0.575446,-0.002640,-0.520961


In [9]:
tag = []
for index, row in df_na.iterrows():
    corr, mon, cc = row['Corr'], row['Monitoring_Site'], row['Clus_Coef']
    if corr>=0.25:
        tag.append('A')
    elif corr<=-0.25 and mon>0 and cc<0:
        tag.append('NA')
    else:
        tag.append('Blank')
df_na['Tag'] = tag
f= lambda x: 1 if x=='A' else (-1 if x=='NA' else 0)
df_na['A_Status'] = df_na['Tag'].apply(f)

c:\program files (x86)\python36-32\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

c:\program files (x86)\python36-32\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [20]:
", ".join(df_na[df_na['Tag']=='Blank']['State_Name'].tolist())

'Illinois, Indiana, Kentucky, Massachusetts, Michigan, Minnesota, Mississippi, New Mexico, New York, North Carolina, North Dakota, Ohio, Pennsylvania, Wisconsin'

In [10]:
from plotly.subplots import make_subplots

def get_data_and_layout(key,title,bar_title):
    if key=='A_Status':
        colorbar = dict(
                title = bar_title,
                tickmode= 'array',
                ticktext = ['Attainment','Borderline','Nonattainment'],
                tickvals = ['0.5','0','-0.5']

            )
    else:
        colorbar = dict(
                title = bar_title
            )
    
    
    data = [ dict(
            type='choropleth',
            colorscale = 'prgn',
            autocolorscale = False,
            locations = df_na['State_Abbre'],
            z = df_na[key].astype(float),
            locationmode = 'USA-states',

            text = df_na['Corr'].astype(str),
            marker = dict(
                line = dict (
                    color = 'rgb(255,255,255)',
                    width = 1
                )
            ),
            colorbar = colorbar
        ) ]

    layout = dict(
            title=go.layout.Title(
            text=title,
            font=dict(
                size = 22,
                color="purple"
            )
        ),
            geo = dict(
                scope='usa',
                projection=dict( type='albers usa' ),
                showlakes = False
            ),
              margin=go.Margin(
            l=0,
            r=0,
            b=0,
#             t=5,
            pad=0
        )
#             annotations=[
#         go.layout.Annotation(
#             x=-0.07,
#             y=0.5,
#             showarrow=False,
#             text="a)",
#             textangle=-90,
#             xref="paper",
#             yref="paper"
#         )
#     ]

        )
    return data,layout

fig = make_subplots(rows=1, cols=3)

keys = ['Monitoring_Site','Clus_Coef','A_Status']
bar_titles = ["Slope","Slope",""]
titles = ['a) Monitoring Sites','b) Clustering Coefficient','c) Attainment & Nonattainment States']
texts = ['a)','b)','c)']
for key, title, bar_title in zip(keys, titles, bar_titles) :
    index = titles.index(title)
    data, layout = get_data_and_layout(key, title, bar_title)
    fig = go.Figure(data=data, layout=layout)
    plotly.offline.plot(fig, filename=key, image='png')

#     fig.show()
#     fig.sav

c:\program files (x86)\python36-32\lib\site-packages\plotly\graph_objs\_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin


c:\program files (x86)\python36-32\lib\site-packages\plotly\offline\offline.py:562: UserWarning:

Your filename `Monitoring_Site` didn't end with .html. Adding .html to the end of your file.

c:\program files (x86)\python36-32\lib\site-packages\plotly\offline\offline.py:562: UserWarning:

Your filename `Clus_Coef` didn't end with .html. Adding .html to the end of your file.

c:\program files (x86)\python36-32\lib\site-packages\plotly\offline\offline.py:562: UserWarning:

Your filename `A_Status` didn't end with .html. Adding .html to the end of your file.



In [15]:
# write df to file
df_write = df_na.reset_index().drop(['index', 'A_Status'], axis = 1)
f = lambda x:'Performing' if x == 'A' else ('Underperforming' if x == 'NA' else 'Borderline')
df_write['Status'] = df_write['Tag'].apply(f)

In [17]:
df_write.drop(['Tag'], axis = 1, inplace = True)

In [18]:
df_write.to_csv("Supplementary_Table_.csv", index = True)

In [19]:
df_write[df_write['Status']=='Underperforming']

,State_Code,State_Abbre,State_Name,Monitoring_Site,Clus_Coef,Corr,Status
4,8,CO,Colorado,0.992341,-0.003787,-0.713072,Underperforming
5,9,CT,Connecticut,0.049021,-0.000891,-0.540416,Underperforming
9,13,GA,Georgia,0.575446,-0.002640,-0.520961,Underperforming
10,16,ID,Idaho,0.148703,-0.005017,-0.494380,Underperforming
17,23,ME,Maine,0.332750,-0.001031,-0.293784,Underperforming
18,24,MD,Maryland,0.042893,-0.000275,-0.768585,Underperforming
25,31,NE,Nebraska,0.037313,-0.000964,-0.491761,Underperforming
28,34,NJ,New Jersey,0.014115,-0.000497,-0.742030,Underperforming
34,40,OK,Oklahoma,0.713973,-0.001711,-0.474798,Underperforming
35,41,OR,Oregon,0.127694,-0.001912,-0.456812,Underperforming
